## Documents

In [1]:
from glob import glob

with open('docs.txt') as fp:
    docs = [d.strip() for d in fp.readlines()]
docs_r = {k:i for i, k in enumerate(docs)}

In [2]:
docs[:20]

['../dataset/IR_dataset/2048.txt',
 '../dataset/IR_dataset/2404.txt',
 '../dataset/IR_dataset/661.txt',
 '../dataset/IR_dataset/1252.txt',
 '../dataset/IR_dataset/726.txt',
 '../dataset/IR_dataset/3029.txt',
 '../dataset/IR_dataset/329.txt',
 '../dataset/IR_dataset/1481.txt',
 '../dataset/IR_dataset/1511.txt',
 '../dataset/IR_dataset/1127.txt',
 '../dataset/IR_dataset/2447.txt',
 '../dataset/IR_dataset/2907.txt',
 '../dataset/IR_dataset/630.txt',
 '../dataset/IR_dataset/3008.txt',
 '../dataset/IR_dataset/494.txt',
 '../dataset/IR_dataset/321.txt',
 '../dataset/IR_dataset/40.txt',
 '../dataset/IR_dataset/3131.txt',
 '../dataset/IR_dataset/2192.txt',
 '../dataset/IR_dataset/2167.txt']

## Loading Dev Dataset

In [3]:
import yaml

In [4]:
with open('../dataset/evaluation_IR.yml', 'r') as f:
    dataset = yaml.safe_load(f)

In [5]:
len(dataset)

150

In [6]:
dataset_dev = {k:v for k, v in list(dataset.items())[-30:]}

In [7]:
list(dataset_dev.keys())

['نظریه اصل موضوعی مجموعه\u200cها  سازگاری و عدم وابستگی در ZFC',
 'نظریه نسبیت  نسبیت عام',
 'نلسون ماندلا  آغاز',
 'نهنگ قاتل  هوش',
 'نوا (دستگاه موسیقی)  گوشه\u200cها',
 'نیروهای محور  اتحاد دانوب، اختلاف بر سر اتریش',
 'هاینریش هیملر  استحکام قدرت',
 'هاینریش هیملر  رابطه با هیتلر',
 'هسته لینوکس',
 'هسته لینوکس  درگیری\u200cهای جامعه توسعه',
 'هسته لینوکس  مدل توسعه',
 'هم\u200cارزی جرم و انرژی  کاربست\u200cپذیری فرمول',
 'هندسه جبری',
 'هوش مصنوعی  تاریخچه',
 'واپاشی هسته\u200cای  پایداری و ناپایداری ایزوتوپ\u200cها',
 'وشمگیر  وضعیت سیاسی-اجتماعی قرن چهارم هجری',
 'ولایت قندهار  تمدن مندیگک',
 'ولفگانگ آمادئوس موتسارت  موتسارت در وین',
 'ونکوور  سیستم حمل و نقل شهری',
 'ونکوور  معماری',
 'پروین اعتصامی',
 'پرچم ایران  پیش از پادشاهی پهلوی\u200cها',
 'پیمان کیوتو  اتحادیه اروپا',
 'چرخه آب  توصیف',
 'چنگیز خان  کودکی',
 'چهاردهمین دالایی لاما  اوان زندگی و سابقه',
 'کارل مارکس  اقتصاد، تاریخ و جامعه',
 'گرجستان  تاریخ',
 'یانی  تأثیرپذیری\u200cهای موسیقایی',
 'یونسکو  فعالیت\u20

In [8]:
dataset_dev['گرجستان  تاریخ']

{'relevant': [388],
 'similar_high': [389, 390, 391, 392, 393, 394],
 'similar_low': [404, 405, 406, 407, 408, 409, 410, 411, 412, 413],
 'similar_med': [395, 364, 396, 397, 398, 399, 400, 401, 402, 403]}

In [50]:
import pandas as pd

queries = pd.Series(list(dataset.keys()))
qrels = [{'query_id':q, 'doc_id':str(d),
          'relevance':3} for idx,q in queries.to_dict().items() for d in dataset[q]['similar_high']]
qrels.extend([{'query_id':q, 'doc_id':str(d),
          'relevance':2} for idx,q in queries.to_dict().items() for d in dataset[q]['similar_med']])
qrels.extend([{'query_id':q, 'doc_id':str(d),
          'relevance':1} for idx,q in queries.to_dict().items() for d in dataset[q]['similar_low']])
qrels.extend([{'query_id':q, 'doc_id':str(dataset[q]['relevant'][0]),
          'relevance':4} for idx,q in queries.to_dict().items()])
qrels = pd.DataFrame(qrels)

In [10]:
qrels

,query_id,doc_id,relevance
0,نظریه اصل موضوعی مجموعه‌ها سازگاری و عدم وابس...,2713,3
1,نظریه اصل موضوعی مجموعه‌ها سازگاری و عدم وابس...,2714,3
2,نظریه اصل موضوعی مجموعه‌ها سازگاری و عدم وابس...,661,3
3,نظریه نسبیت نسبیت عام,1617,3
4,نظریه نسبیت نسبیت عام,1619,3
...,...,...,...
727,چهاردهمین دالایی لاما اوان زندگی و سابقه,2968,4
728,کارل مارکس اقتصاد، تاریخ و جامعه,365,4
729,گرجستان تاریخ,388,4
730,یانی تأثیرپذیری‌های موسیقایی,1748,4


## POS Tagger

In [11]:
import tensorflow
from tensorflow.keras.models import load_model

In [12]:
with open('pos/words.txt') as fp:
    words_all = [line.strip() for line in fp.readlines()]
word2int = {k:i for i, k in enumerate(words_all)}
VOCAB_SIZE = len(word2int) + 1

In [13]:
with open('pos/tags.txt') as fp:
    tags_all = [line.strip() for line in fp.readlines()]
tag2int = {k:i for i, k in enumerate(tags_all)}
TAGS_NO = len(tag2int)

In [14]:
from tensorflow.keras.models import load_model
pos_tagger = load_model('models/pos_lstm_3.h5')

In [15]:
pos_tagger.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 100)           7840700   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 160)          115840    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 50, 34)           5474      
 ibuted)                                                         
                                                                 
Total params: 7,962,014
Trainable params: 7,962,014
Non-trainable params: 0
_________________________________________________________________


In [16]:
SEQ_LEN = 50

In [17]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [18]:
allowed_chars = [
    'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش',
    'ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ى','ي','٠','١','٢','٣', '٤', '٥', '٦', '٧',
    '٨', '٩', 'چ', 'ژ', 'ک', 'گ', 'ھ', 'ی', '۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹',
    '\u200c', '\u200d', '\u200e', '\u200f',# 'ٹ', 'څ', 'ڈ', 'ڑ', 'ڕ',
    'ﭼ', 'ﯽ', 'ﯾ', 'ﯿ', 'ﷲ', 'ﺀ', 'ﺄ', 'ﺆ', 'ﺋ', 'ﺎ', 'ﺑ', 'ﺔ', 'ﺗ', 'ﺘ', 'ﺧ', 'ﺪ', 'ﺮ', 'ﺳ', 'ﺴ', 'ﺿ',
    'ﻋ','ﻌ', 'ﻗ', 'ﻠ', 'ﻣ', 'ﻨ', 'ﻼ', '￼', 'پ',]

trans_chars = [
    'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٓ', 'ٔ',
]

In [19]:
import re
import nltk
from hazm import Normalizer

normalizer = Normalizer()

def normalize_text(text):
    text = normalizer.normalize(text)
    text = ' '.join([a.strip() for a in re.split("([۰-۹]+)", text) if a])
    text = re.sub('[' + ''.join(trans_chars) + ']', '', text)
    text = re.sub('[^' + ''.join(allowed_chars) + ']', ' ', text)
#     text = re.sub('ئ', 'ی', text)
    text = re.sub('ء', '', text)
    text = re.sub('[\s]+', ' ', text)
    return text

def encode_text(text):
    text = normalize_text(text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens, [word2int[word] if word in word2int else word2int['[UNK]'] for word in tokens]

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def pos_tag(text):
    test_X = []
    test_words = []

    text_words, text_tokens = encode_text(text)
    words_list = list(chunks(text_words, SEQ_LEN))
    tokens_list = list(chunks(text_tokens, SEQ_LEN))
    test_X += tokens_list
    test_words += words_list
    test_X = pad_sequences(test_X, maxlen=SEQ_LEN, padding='post')
    pred_outs = pos_tagger.predict(test_X)
    pred_args = np.argmax(pred_outs, axis=2)
    pred_tags = []
    for i, pred in enumerate(pred_args):
        cur_tags = [tags_all[i] if i in range(len(tags_all)) else 'UNK' for i in pred]
        cur_pairs = list(zip(test_words[i], cur_tags))
        pred_tags += cur_pairs
    return pred_tags

### POS Tag Test

In [48]:
from random import sample

with open('../dataset/IR_dataset/78.txt') as fp:
    text = fp.read()
    text = re.sub('\n', ' ', text)
    print(pos_tag(text))

[('اصل', 'N_SING'), ('موضوع', 'N_SING'), ('اجتماع', 'N_SING'), ('بیان', 'N_SING'), ('می\u200cکند', 'V_PRS'), ('۱', 'N_SING'), ('یا', 'CON'), ('به', 'P'), ('بیان', 'N_SING'), ('دیگر', 'ADJ'), ('برای', 'P'), ('هر', 'DET'), ('دسته', 'N_SING'), ('دلخواه', 'ADJ'), ('از', 'P'), ('مجموعه\u200cها', 'N_PL'), ('مجموعه\u200cای', 'N_SING'), ('وجود', 'N_SING'), ('دارد', 'V_PRS'), ('که', 'CON'), ('شامل', 'ADJ'), ('همه', 'PRO'), ('عناصری', 'N_PL'), ('است', 'V_PRS'), ('که', 'CON'), ('حداقل', 'ADV'), ('به', 'P'), ('یکی', 'PRO'), ('از', 'P'), ('مجموعه\u200cهای', 'N_PL'), ('دسته', 'N_SING'), ('مفروض', 'N_SING'), ('متعلق', 'ADJ'), ('باشند', 'V_SUB'), ('به', 'P'), ('بیان', 'N_SING'), ('دیگر', 'ADJ'), ('اگر', 'CON'), ('دسته\u200cای', 'N_SING'), ('از', 'P'), ('مجموعه\u200cها', 'N_PL'), ('باشد', 'V_SUB'), ('مجموعه\u200cای', 'N_SING'), ('چون', 'CON'), ('وجود', 'N_SING'), ('دارد', 'V_PRS'), ('که', 'CON'), ('اگر', 'CON'), ('موجود', 'ADJ'), ('باشد', 'V_SUB'), ('به\u200cطوری\u200cکه', 'N_SING'), ('آنگاه', 'ADV'), 

## Word2Vec

In [53]:
import numpy as np

word2vec = np.load("w2v/word2vec2.npy")

In [54]:
with open('w2v/vocab2.txt') as fp:
    vocab = [l.strip() for l in fp.readlines()]
vocab_r = {k:i for i, k in enumerate(vocab)}

In [55]:
len(vocab), word2vec.shape

(75011, (75011, 200))

In [56]:
EMBED_DIM = 200

In [57]:
def get_word2vec(word):
    return word2vec[vocab_r[word]] if word in vocab_r else np.zeros((EMBED_DIM,))

## POS Tag Weights

In [58]:
tag_w = {
    'PAD': 0,
    'ADJ': 1,
    'ADJ_CMPR': 1,
    'ADJ_INO': 1,
    'ADJ_SUP': 1,
    'ADJ_VOC': 1,
    'ADV': 0.5,
    'ADV_COMP': 0.5,
    'ADV_I': 0.5,
    'ADV_LOC': 0.5,
    'ADV_NEG': 0.5,
    'ADV_TIME': 0.5,
    'CLITIC': 0,
    'CON': 0,
    'DELM': 0,
    'DET': 0,
    'FW': 2,
    'INT': 0,
    'NUM': 2,
    'N_PL': 6,
    'N_SING': 6,
    'N_VOC': 6,
    'P': 0,
    'PREV': 0,
    'PRO': 0,
    'SYM': 0,
    'UNK': 0,
    'V_AUX': 0,
    'V_IMP': 0,
    'V_PA': 0,
    'V_PP': 0,
    'V_PRS': 0,
    'V_SUB': 0
}

## Tf-Idf Vectorization

In [59]:
from scipy.sparse import load_npz

tfidf = load_npz('tfidf/tfidf.npz')

tfidf_words = []
with open('tfidf/words.txt') as fp:
    tfidf_words = [w.strip() for w in fp.readlines()]
tfidf_words_r = {k:i for i, k in enumerate(tfidf_words)}

In [106]:
tfidf[docs_r['../dataset/IR_dataset/2048.txt'], tfidf_words_r['پودر']]

0.30680919856295225

In [60]:
def get_tfidf(doc, word):
    return tfidf[docs_r[doc], tfidf_words_r[word]] if word in tfidf_words_r else 0

## Document Embedding

In [61]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [62]:
def embed_doc(doc, text, tag_wi=tag_w, has_tfidf=True):
    pos_tags = pos_tag(text)
    vectors = np.zeros((len(pos_tags), EMBED_DIM), dtype=np.float64)
    weights = np.zeros((len(pos_tags),), dtype=np.float64)
    i = 0
    for word, tag in pos_tags:
        vectors[i] = get_word2vec(word)
        weights[i] = tag_wi[tag]
        if has_tfidf:
            weights[i] += get_tfidf(doc, word) # To be tuned
        i += 1
        
    s = softmax(weights)
    return np.dot(s, vectors)

In [63]:
def cos_sim(a, b):
    na = np.linalg.norm(a)
    nb =np.linalg.norm(b) 
    return np.dot(a, b) / (na * nb) if na > 0 and nb > 0 else 0

In [64]:
def embed_query(query):
    pos_tags = pos_tag(query)
    vectors = np.zeros((len(pos_tags), EMBED_DIM), dtype=np.float64)
    weights = np.zeros((len(pos_tags),), dtype=np.float64)
    i = 0
    for word, tag in pos_tags:
        vectors[i] = get_word2vec(word)
        weights[i] = 1 / len(pos_tags)
        i += 1
    return np.dot(weights, vectors)

In [65]:
def embed_docs(tag_wi=tag_w, has_tfidf=True):
    doc_vectors = np.zeros((len(docs), EMBED_DIM), dtype=np.float64)
    for i, doc in enumerate(docs):
        with open(doc) as fp:
            text = fp.read()
            text = re.sub('\n', ' ', text)
        try:
            doc_vectors[i] = embed_doc(doc, text, tag_wi, has_tfidf)
        except:
            print('Error', doc, 'at', i, 'occurred.')
    return doc_vectors

## Evaluation Metrics

In [34]:
import ir_measures as IRm

In [35]:
MRR = IRm.measures.MRR()
def mrr_measure(qrels, outputs):
    ret = pd.DataFrame(outputs, columns=['query_id', 'doc_id', 'score'])
    return MRR.calc_aggregate(qrels[qrels.relevance == 4], ret)

In [36]:
def map_measure(qrels, outputs):
    ret = pd.DataFrame(outputs, columns=['query_id', 'doc_id', 'score'])
    return np.mean([IRm.measures.AP(rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret) for level in range(1,4+1)])

In [37]:
def p_measure(qrels, outputs):
    ret = pd.DataFrame(outputs, columns=['query_id', 'doc_id', 'score'])
    return np.mean([IRm.measures.P(cutoff=k, rel=level).\
                    calc_aggregate(qrels[qrels.relevance == level], ret)\
                  for k,level in zip([1,11,21,31],range(1,4+1))])

## Retrieve & Evaluate

In [66]:
doc_vectors = embed_docs()

Error ../dataset/IR_dataset/2885.txt at 569 occurred.


In [67]:
def get_doc_id(doc):
    return int(doc.split('/')[-1].split('.')[0])

def retrieve_docs(query, doc_vectors):
    q_vec = embed_query(query)
    scores = list(zip([query for _ in range(len(docs))],
                      [str(get_doc_id(d)) for d in docs], 
                      np.dot(doc_vectors, q_vec)))
    return sorted(scores, key=lambda x: -x[2])

In [68]:
outputs = []

for query in dataset:
    #print('Q:', query)
    #print(dataset_dev[query])
    retrieved = retrieve_docs(query, doc_vectors)[:100]
    outputs += retrieved
print('MRR:', mrr_measure(qrels, outputs))
print('MAP:', map_measure(qrels, outputs))
print('P@k:', p_measure(qrels, outputs))

MRR: 0.2402969847760753
MAP: 0.16490216535458788
P@k: 0.07264790764790764


## Hyperparameter Tuning

In [38]:
for w in [3, 6, 9, 12, 18]:
    tag_w['N_PL'] = tag_w['N_SING'] = w
    print('Tag weight:', w, 'Tf-idf: T')
    doc_vectors = embed_docs(tag_w, has_tfidf=True)
    outputs = []
    for query in dataset_dev:
        retrieved = retrieve_docs(query, doc_vectors)[:30]
        outputs += retrieved
    print('MRR:', mrr_measure(qrels, outputs))
    print('MAP:', map_measure(qrels, outputs))
    print('P@k:', p_measure(qrels, outputs))
    
    print('Tag weight:', w, 'Tf-idf: F')
    doc_vectors = embed_docs(tag_w, has_tfidf=False)
    outputs = []
    for query in dataset_dev:
        retrieved = retrieve_docs(query, doc_vectors)[:30]
        outputs += retrieved
    print('MRR:', mrr_measure(qrels, outputs))
    print('MAP:', map_measure(qrels, outputs))
    print('P@k:', p_measure(qrels, outputs))

Tag weight: 3 Tf-idf: T
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.21411914882503114
MAP: 0.15861841426376194
P@k: 0.07136689475399151
Tag weight: 3 Tf-idf: F
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.2231447447826758
MAP: 0.1426937241899583
P@k: 0.06707396546106223
Tag weight: 6 Tf-idf: T
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.22087218337218334
MAP: 0.15974856968562778
P@k: 0.07255737094446771
Tag weight: 6 Tf-idf: F
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.2175260178306155
MAP: 0.14092246351854398
P@k: 0.06925475957734022
Tag weight: 9 Tf-idf: T
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.22007853257853255
MAP: 0.1586301014453418
P@k: 0.07179979518689195
Tag weight: 9 Tf-idf: F
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.21345397875857644
MAP: 0.13813564618263247
P@k: 0.06925475957734022
Tag weight: 12 Tf-idf: T
Error ../dataset/IR_dataset/2885.txt at 569 occurred.
MRR: 0.

In [ ]:
sorted(list(zip([int(d.split('/')[-1].split('.')[0]) for d in docs], 
                np.dot(doc_vectors, q_vec) / (np.linalg.norm(q_vec) * np.linalg.norm(doc_vectors, axis=1)))), 
                key=lambda x: -x[1])[:20]

In [ ]:
sorted(list(zip([int(d.split('/')[-1].split('.')[0]) for d in docs], np.dot(doc_vectors, q_vec))), 
       key=lambda x: -x[1])[:20]